In [67]:
import numpy
import pandas as pd 
import nltk
import string
import re
import os
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

In [68]:
art_df = pd.read_csv('C:/Users/Powerhouse/Documents/GitHub/Project-High/article-database.csv')
print(art_df.shape)
art_df.head()

(11649, 6)


,Unnamed: 0,Name,Url,Text,Tags,Claps
0,0,None,https://medium.com/@elektra.sunn/cheshire-moon...,NaN,[],[]
1,0,None,https://medium.com/@nicolascole77/if-youre-doi...,NaN,[],[]
2,0,The Ultimate Travel Workout and Diet Plans by ...,https://medium.com/@garrettgee/the-bucket-list...,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...",[]
3,0,Building a Responsive Image Gallery,https://medium.com/backticks-tildes/building-a...,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']",[]
4,0,When they’re already “forever” for you,https://medium.com/@krisgage/when-theyre-alrea...,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...",[]


In [69]:
model_db = art_df.drop([art_df.columns[0], art_df.columns[1], art_df.columns[2], art_df.columns[5]], axis=1)
model_db.head()

,Text,Tags
0,NaN,[]
1,NaN,[]
2,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
3,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
4,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."


In [70]:
model_db.isnull().sum()

Text    674
Tags      0
dtype: int64

In [71]:
model_db_clean = model_db.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop=True)
model_db_clean.head()

,Text,Tags
0,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
1,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
2,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."
3,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']"
4,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']"


In [72]:
sum = 0
for i in range(0, model_db_clean.shape[0]):
    sum = sum + len(model_db_clean.Text[i])
sum/model_db_clean.shape[0]

6891.549066059226

In [73]:
# from ast import literal_eval
def func(raw_tags):
    raw_split = raw_tags[1:-1].split(',')
    num_tags = len(raw_split)
    tags_clean = []
    tags_clean.append(raw_split[0][1:-1])
    for i in range(1, num_tags):
       tags_clean.append(raw_split[i][2:-1])
    return tags_clean

model_db_clean['Tags_clean'] = model_db_clean['Tags'].apply(lambda x: func(x))

# model_db_clean.loc[:,'Tags'] = model_db_clean.loc[:,'Tags'].apply(lambda x: literal_eval(x))
print(type(model_db_clean['Tags']))
model_db_clean.head()

<class 'pandas.core.series.Series'>


,Text,Tags,Tags_clean
0,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...","[Fitness, Travel, Diet, Exercise, Vacation]"
1,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']","[Design, Data, Growth, CSS, Web Design]"
2,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...","[Love, Relationships, Dating, Men, Women]"
3,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']","[Publishing, Apple, Apple]"
4,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']","[Swift, iOS, Apple]"


In [74]:
multi_label_transform = MultiLabelBinarizer()
multi_label_transform.fit(model_db_clean.Tags_clean)

y = multi_label_transform.transform(model_db_clean.Tags_clean)
print(y.shape, model_db_clean.shape)

(10975, 886) (10975, 3)


In [75]:
cols = []
for i in list(multi_label_transform.classes_):     
    cols.append(i)
cols.append('Text')

prepd_db = pd.DataFrame()

prepd_db.loc[:, 'Text'] = model_db_clean.loc[:, 'Text']
for i in range(0, y.shape[1]):
        prepd_db.loc[:, i+1] = y[:, i]

def clean_text(text):
    text = re.sub("\'", "", text) 
    text = re.sub("[^a-zA-Z]"," ",text) 
    text = ' '.join(text.split()) 
    text = text.lower() 
    
    return text

prepd_db['Text_clean'] = prepd_db['Text'].apply(lambda x: clean_text(x))
prepd_db.drop(['Text'], axis=1, inplace=True)

prepd_db.columns = cols
prepd_db.drop([''], axis=1, inplace=True)

prepd_db.head()

,2016 Election,2017,AI,API,AWS,Abortion,Abuse,Academia,Accessibility,Accounting,...,YouTube,Youth,Zen,eBooks,iOS,iOS App Development,iPad,iPhone,日本語,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introducing the bucket list family hi we are t...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in this post you would learn to build a basic ...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,because love that s light can last that long a...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,by jean louis gass e a delayed monday note due...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,today i was integrating fcm in my ios app i sa...


In [76]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

prepd_db['Text'] = prepd_db['Text'].apply(lambda x: remove_stopwords(x))
prepd_db.head()

,2016 Election,2017,AI,API,AWS,Abortion,Abuse,Academia,Accessibility,Accounting,...,YouTube,Youth,Zen,eBooks,iOS,iOS App Development,iPad,iPhone,日本語,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introducing bucket list family hi bucket list ...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,post would learn build basic image gallery fle...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,love light last long past partners frankly pro...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jean louis gass e delayed monday note due logi...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,today integrating fcm ios app saw word method ...


In [77]:
for i in range(0, 885):
    x = prepd_db.iloc[:, i].value_counts()
    if x[1] > 100:
        print(x)

0    10661
1      314
Name: AI, dtype: int64
0    10850
1      125
Name: Android, dtype: int64
0    10829
1      146
Name: Apple, dtype: int64
0    10851
1      124
Name: Architecture, dtype: int64
0    10815
1      160
Name: Art, dtype: int64
0    10818
1      157
Name: Artificial Intelligence, dtype: int64
0    10865
1      110
Name: Big Data, dtype: int64
0    10781
1      194
Name: Bitcoin, dtype: int64
0    10867
1      108
Name: BlackLivesMatter, dtype: int64
0    10385
1      590
Name: Blockchain, dtype: int64
0    10836
1      139
Name: Blog, dtype: int64
0    10868
1      107
Name: Blogging, dtype: int64
0    10682
1      293
Name: Books, dtype: int64
0    10868
1      107
Name: Branding, dtype: int64
0    10661
1      314
Name: Business, dtype: int64
0    10703
1      272
Name: Career, dtype: int64
0    10831
1      144
Name: Careers, dtype: int64
0    10852
1      123
Name: College, dtype: int64
0    10862
1      113
Name: Computer Science, dtype: int64
0    10686
1      289

In [112]:
y = []
for i in range(0, 885):
    x = prepd_db.iloc[:, i].value_counts()
    if x[1] > 50:
        y.append(x.name)      
  
model_final_data = prepd_db[y]
model_final_data['Text'] = prepd_db['Text']

model_final_data.head()

,AI,Advertising,Advice,Analytics,Android,Android App Development,Apple,Apps,Architecture,Art,...,White Privilege,Wisdom,Women,WordPress,Work,Writing,Writing Tips,iOS,iOS App Development,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introducing bucket list family hi bucket list ...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,post would learn build basic image gallery fle...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,love light last long past partners frankly pro...
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,jean louis gass e delayed monday note due logi...
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,today integrating fcm ios app saw word method ...


In [113]:
model_final_data.to_csv('model-data.csv')

In [105]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=1000)

for x in range(0, len(model_final_data.columns)-1):
    xtrain, xval, ytrain, yval = train_test_split(model_final_data['Text'], model_final_data.iloc[:, x], test_size=0.2, random_state=9)

    xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
    xval_tfidf = tfidf_vectorizer.transform(xval)

    lr = LogisticRegression()
    lr.fit(xtrain_tfidf, ytrain)

    joblib.dump(lr, 'model_pickle_files/model_feature_' + str(x) + '.pkl')

In [111]:
for i in range(0, 121):
    feature_number = i
    job_lr = joblib.load('model_pickle_files/model_feature_' + str(feature_number) + '.pkl')

    _te = 'fortunately libraries scikit learn easy implement hundreds machine learning algorithms python easy often need underlying knowledge model works order use knowing details necessary still helpful idea machine learning model works hood lets us diagnose model underperforming explain makes decisions crucial want convince others trust models article look build use random forest python addition seeing code try get understanding model works random forest made many decision trees start understanding single decision tree makes classifications simple problem work way using random forest real world data science problem complete code article available jupyter notebook github note article originally appeared enlight community driven open source platform tutorials looking study machine learning decision tree building block random forest intuitive model think decision tree series yes questions asked data eventually leading predicted class continuous value case regression interpretable model makes classifications much like ask sequence queries available data arrive decision ideal world technical details decision tree questions data formed cart algorithm decision tree built determining questions called splits nodes answered lead greatest reduction gini impurity means decision tree tries form nodes containing high proportion samples data points single class finding values features cleanly divide data classes talk low level detail gini impurity later first let build decision tree understand high level'
    text_tfidf = tfidf_vectorizer.transform([_te])
    y_pred = job_lr.predict(text_tfidf)
    if y_pred[0] == 1:
        print(model_final_data.columns[i])

Machine Learning
